# Automatic comparison of STIX and AIA images

The aim of this script is to take as input the FITS files with STIX images and then downloading, processing and doing cutout of AIA 1600 or AIA 1700 images

This code is thought to be working with the REPROJ virtual environment

### Imports and functions

In [1]:
#import sys
#sys.path.insert(1, '/home/afbattaglia/Software/idl_lib/STIX-science/Python')

import astropy.units as u
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from aiapy.calibrate import fix_observer_location, normalize_exposure, register, update_pointing
from astropy.coordinates import SkyCoord
from astropy.time import Time
#from rotated_map import *
from sunpy.coordinates import Helioprojective
from sunpy.coordinates.ephemeris import get_earth
from sunpy.coordinates.frames import HeliocentricEarthEcliptic, HeliographicStonyhurst
from sunpy.map import Map, all_coordinates_from_map, make_fitswcs_header
from sunpy.net import attrs as a
from sunpy.net import Fido

In [2]:
plt.rcParams.update({'font.size': 14})

In [3]:
def download_aia(peak_time, wavelength, email, prep = True,
                           folder_store=''):
    """
    Download the AIA map
    """

    # JSOC email
    #jsoc_email = os.environ[email]

    # Define starting and ending time of the interval
    # for searching the AIA map (default 12 seconds)
    dt = 12
    time_start = Time(peak_time)-dt*u.s
    time_end = Time(peak_time)+dt*u.s

    # Search for the data
    query = Fido.search(#a.Instrument.aia,
                    #a.Physobs.intensity,
                    #a.jsoc.Series.aia_lev1_uv_24s,
                    #a.jsoc.Notify(jsoc_email),
                    #a.jsoc.Segment.image,
                    #a.Wavelength(wavelength*u.angstrom),
                    #a.Time(time_start, time_end)
                    a.Time(time_start, time_end),
                    a.Wavelength(wavelength*u.angstrom),
                    a.Sample(48*u.s),
                    a.jsoc.Series.aia_lev1_uv_24s,
                    a.jsoc.Notify(email),
                    a.jsoc.Segment.image,
                    )
    
    # Download the data
    file_downloaded = Fido.fetch(query[0][0], path=folder_store)
    #if file_downloaded == []:
    #    print('No data found. We try to download the data from ROB now...')
    #    file_downloaded = Fido.fetch(query[0][0], site='ROB', path=folder_store)
    map = Map(file_downloaded)

    # If perp == True, then process the map
    if prep == True:
        map = process_aia(map)

    return map

In [4]:
def process_aia(map):
    """
    Process the AIA map to level 1.5 and
    update the pointing information in the header.
    """

    # Calibrate the map
    map = register(update_pointing(map))
    map = normalize_exposure(map)
    map = fix_observer_location(map)

    return map

In [5]:
def reproject_aia2so(aia_map, stix_map, out_shape = (1024, 1024)):
    """
    Reproject the AIA map to the Solar Orbiter view
    """

    # Get the coordinates of the center of the STIX map in arcsec
    all_coords = all_coordinates_from_map(stix_map)
    # get the coordinates of the center of the map
    center = SkyCoord(all_coords.Tx.mean(), all_coords.Ty.mean(), frame=stix_map.coordinate_frame)
    mapcenter = [center.Tx.value, center.Ty.value]

    # Define the reference coordinate
    ref_coord = SkyCoord(
                     mapcenter[0]*u.arcsec, mapcenter[1]*u.arcsec,
                     #0*u.arcsec, 0*u.arcsec,
                     frame='helioprojective', 
                     obstime=stix_map.date, 
                     observer=stix_map.observer_coordinate)


    # Reference pixel in the AIA map
    #ref_pix = stix_map.world_to_pixel(center)

    # Distance AIA and SO to the Sun
    dist_aia = aia_map.observer_coordinate.radius.value
    dist_so  = stix_map.observer_coordinate.radius.value
    factor = dist_aia / dist_so
    pixel_size = [float(aia_map.scale[0]/u.arcsec*u.pixel) * factor, 
                  float(aia_map.scale[1]/u.arcsec*u.pixel) * factor]

    # Create a FITS WCS header for the reference coordinate and frame
    header = make_fitswcs_header(
        out_shape,
        ref_coord,
        scale = pixel_size*u.arcsec/u.pixel
        #reference_pixel = ref_pix*u.pix
        )

    # Reproject the AIA map
    map = aia_map.reproject_to(header)
    
    return map, mapcenter

In [6]:
def hgs2hpc(hsg_coord, date, observer):
    '''
    This function takes as an input the Heliographic Stonyhurts coordinates
    and returns the Helioprojective coordinates as observed by Earth.
     - hsg_coord must be a 2D array [lon (deg), lat (deg)]
     - date of the observation
     - observer must be the result of map.observer_coordinate (of an AIA map)
    '''

    # SkyCoord of the input coordinates
    hsg_skycoord = SkyCoord(hsg_coord[0]*u.deg, 
                            hsg_coord[1]*u.deg, 
                            #obstime=date,
                            frame=HeliographicStonyhurst(obstime=date))

    # Transform to Helioprojective cartesian
    hpc_coord = hsg_skycoord.transform_to(Helioprojective(obstime=date,
                                          observer=observer))

    return hpc_coord

### Parameters

In [7]:
# Folder containing the STIX maps
folder_stix = '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/STIX-maps/'

# Folder where to store the reprojected AIA maps
folder_store_rmaps = '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/AIA-rmaps/'

# Folder where to store the AIA full-disk maps
folder_store = '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/Data/SDO_AIA/full-disk/'

# Folder where to store the overview plots
folder_plots = '/home/afbattaglia/Software/STIX_estimate_pointing_error/overview-images/'

# Email associated with the JSOC account
# Not registered yet? Go to: http://jsoc.stanford.edu/ajax/register_email.html
email = 'andrea-battaglia@ethz.ch'

# Wavelengths of the AIA maps to download
wavelength = 1600

# FOV of the AIA maps
fov = [400, 400]

# Final FOV of the reprojected AIA maps with STIX
fov_plot = [200, 200]

# Path to the Frederic's table
frederic_csv = '/home/afbattaglia/Software/STIX_estimate_pointing_error/Flare-location_test-table.csv'

### Main code

In [8]:
# Get the list of STIX maps
list_stix = glob.glob(folder_stix+'*')
list_stix.sort()
list_stix

['/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/STIX-maps/2021-05-07T18:50_2105070034_mem.fits',
 '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/STIX-maps/2021-09-23T15:24_2109230075_mem.fits',
 '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/STIX-maps/2021-10-09T06:28_2110090002_mem.fits',
 '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/STIX-maps/2022-01-20T05:58_2201208962_mem.fits',
 '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/STIX-maps/2022-03-02T17:35_2203026385_mem.fits',
 '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/STIX-maps/2022-03-11T22:26_2203118542_mem.fits',
 '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_location_comparison/STIX-maps/2022-03-14T08:34_2203143075_mem.fits',
 '/home/afbattaglia/Documents/ETHZ/PhD/Codes/automatic_STIX-AIA_locat

In [9]:
# Number of STIX maps
n_stix_maps = len(list_stix)

In [10]:
# Open Frederic's table to get the flare locations as seen from Earth
df = pd.read_csv(frederic_csv)

# Store variables of interests
peak_times_so = list(df.iloc[1:,0])
flare_loc_earth_string = list(df.iloc[1:,2])
uid = list(df.iloc[1:,3])
start_time_so = list(df.iloc[1:,4])
end_time_so = list(df.iloc[1:,5])
energy_low  = list(df.iloc[1:,6])
energy_high = list(df.iloc[1:,7])

flare_loc_earth_string, uid

(['N17E78',
  'S29E16',
  'N18E08',
  'N08W76',
  'N15E29',
  'S27W63',
  'N23E03',
  'N23W10',
  'N23W17',
  'N14W04',
  'N15W24',
  'N13W68',
  'S30W72',
  'S30W78',
  'S29W84',
  'S30W62'],
 ['2105070034',
  '2109230075',
  '2110090002',
  '2201208962',
  '2203026385',
  '2203118542',
  '2203143075',
  '2203159015',
  '2203159363',
  '2203282437',
  '2203293826',
  '2204023786',
  '2204176813',
  '2204185406',
  '2204184714',
  '2208271589'])

In [11]:
# Number of flares
nflares = len(flare_loc_earth_string)
nflares

16

In [12]:
# Transform the locations from HGS string to HPC coordinates
### This should be done by looping on the STIX maps,
### once we have all UIDs associated with each map
flare_loc_earth_hpc = np.zeros([nflares, 2])
for i in range(nflares):

    # Convert the coordinate from string to a 2D array
    sign_ns = 1
    sign_ew = 1
    if flare_loc_earth_string[i][0] == 'S': sign_ns = -1
    if flare_loc_earth_string[i][3] == 'E': sign_ew = -1
    coord_ns = float(flare_loc_earth_string[i][1:3])
    coord_ew = float(flare_loc_earth_string[i][4:6])
    lon = (sign_ew * coord_ew) #* u.deg
    lat = (sign_ns * coord_ns) #* u.deg

    #print(lon, lat)
    #print(coord_ew, coord_ns)

    # Get the Earth coordinates in hgs
    earth_hgs = get_earth(Time(peak_times_so[i]).isot)

    # Transform the coordinates of the flare location from hgs to hpc
    hpc_coord = hgs2hpc([lon, lat], peak_times_so[i], earth_hgs)

    # Store the coordinates
    flare_loc_earth_hpc[i,0] = hpc_coord.Tx.value
    flare_loc_earth_hpc[i,1] = hpc_coord.Ty.value

In [13]:
flare_loc_earth_hpc

array([[-889.80746491,  289.04164184],
       [-231.31490321, -560.16266457],
       [-127.6792824 ,  197.13307983],
       [ 937.69791738,  155.83218566],
       [-454.93013275,  352.79021554],
       [ 768.18179667, -386.60096636],
       [ -46.67415519,  486.99446754],
       [ 154.80476785,  484.96520312],
       [ 260.58323088,  481.56390308],
       [  65.33331533,  341.47926403],
       [ 378.91765872,  346.72921296],
       [ 868.43950631,  254.55684067],
       [ 788.15276504, -452.25147955],
       [ 810.18725869, -460.01565837],
       [ 831.48478636, -453.31219746],
       [ 726.99243056, -519.42707845]])

In [14]:
# Text parameters
x_txt = 0.04
y_txt = 0.96
dy = 0.03

In [15]:
# Loop on all flares in the list
for this_el in range(nflares):

    # Get the variables of the current flare
    this_uid = uid[this_el]
    this_peak_time = peak_times_so[this_el]
    this_flare_loc_earth_hpc = flare_loc_earth_hpc[this_el,:]
    this_time_range = [start_time_so[this_el], end_time_so[this_el]]
    this_energy_range = [energy_low[this_el], energy_high[this_el]]
    
    print()
    print('Flare ID: '+str(this_uid))
    print()

    # Find the STIX map containing this_uid
    for this_map in range(n_stix_maps):
        if this_uid in list_stix[this_map]:
            this_stix_map = list_stix[this_map]
            break

    # Open the first STIX map
    stix_map = Map(this_stix_map)

    # Get the light travel time correction from the FITS header
    ltt = stix_map.meta['EAR_TDEL']
    #print('Light travel time correction: '+str(ltt)+' s')

    # Time at Earth
    time_earth = Time(this_peak_time) + ltt*u.s

    # Download the AIA map
    aia_map = download_aia(time_earth, wavelength, email, folder_store=folder_store)

    # Data providers can set the radius at which emission in the map is 
    # assumed to have come from. Most maps use a default value for 
    # photospheric radius, but some maps (as AIA maps) are set to a slightly 
    # different value. A mismatch in solar radius means a reprojection will 
    # not work correctly on pixels near the limb. This can be prevented by 
    # modifying the values for rsun on one map to match the other.
    aia_map.meta['rsun_ref'] = stix_map.meta['rsun_ref']

    # Extract the submap according to the flare location as seen from Earth
    bottom_left = SkyCoord((this_flare_loc_earth_hpc[0]-fov[0]/2)*u.arcsec,
                           (this_flare_loc_earth_hpc[1]-fov[1]/2)*u.arcsec,
                            frame=aia_map.coordinate_frame)
    top_right = SkyCoord((this_flare_loc_earth_hpc[0]+fov[0]/2)*u.arcsec,
                         (this_flare_loc_earth_hpc[1]+fov[1]/2)*u.arcsec,
                          frame=aia_map.coordinate_frame)
    submap = aia_map.submap(bottom_left, top_right=top_right)

    # Reproject the AIA map to the STIX view
    rmap_aia, stix_center = reproject_aia2so(submap, stix_map)

    # Save the reprojected map
    rmap_aia.save(folder_store_rmaps+'reprojected-AIA'+str(wavelength)+'_'+str(this_peak_time)+'_'+str(this_uid)+'.fits')

    # PLOT
    bottom_left_plot = SkyCoord((stix_center[0]-fov_plot[0]/2)*u.arcsec,
                                (stix_center[1]-fov_plot[1]/2)*u.arcsec,
                                 frame=rmap_aia.coordinate_frame)
    top_right_plot = SkyCoord((stix_center[0]+fov_plot[0]/2)*u.arcsec,
                              (stix_center[1]+fov_plot[1]/2)*u.arcsec,
                               frame=rmap_aia.coordinate_frame)
    sub_rmap_aia = rmap_aia.submap(bottom_left_plot, top_right=top_right_plot)
    sub_stix_map = stix_map.submap(bottom_left_plot, top_right=top_right_plot)

    fig = plt.figure(figsize=(10,10), dpi=100, facecolor='white')
    ax = plt.subplot(projection=sub_rmap_aia)
    sub_rmap_aia.plot(axes=ax)
    sub_stix_map.draw_contours([50,70,90]*u.pct, axes=ax, colors='red')
    lon = ax.coords[0]
    lon.display_minor_ticks(True)
    lat = ax.coords[1]
    lat.display_minor_ticks(True)
    lon.grid(color='black', alpha=1, linestyle='solid')
    lat.grid(color='black', alpha=1, linestyle='solid')
    ax.set_title(aia_map.meta['date-obs']+'   Earth UT')

    ax.text(x_txt, y_txt, 'Flare ID: '+str(this_uid),
            verticalalignment='top', horizontalalignment='left',
            transform=ax.transAxes,
            color='red', fontsize=15)
    ax.text(x_txt, y_txt-dy, 'Time range (SO UT): '+this_time_range[0]+' - '+this_time_range[1],
            verticalalignment='top', horizontalalignment='left',
            transform=ax.transAxes,
            color='red', fontsize=15)
    ax.text(x_txt, y_txt-2*dy, 'Energy range (keV): '+this_energy_range[0]+' - '+this_energy_range[1],
            verticalalignment='top', horizontalalignment='left',
            transform=ax.transAxes,
            color='red', fontsize=15)
    ax.text(x_txt, y_txt-3*dy, 'Contour levels: [50,70,90]%',
            verticalalignment='top', horizontalalignment='left',
            transform=ax.transAxes,
            color='red', fontsize=15)
    ax.text(x_txt, y_txt-4*dy, 'Algorithm: MEM_GE',
            verticalalignment='top', horizontalalignment='left',
            transform=ax.transAxes,
            color='red', fontsize=15)

    plt.savefig(folder_plots+'AIA'+str(wavelength)+'-STIX_'+str(this_peak_time)+'_'+str(this_uid)+'.png')
    #plt.show()
    plt.close()


Flare ID: 2105070034

Export request pending. [id=JSOC_20230721_346, status=2]
Waiting for 0 seconds...
1 URLs found for download. Full request totalling 8MB


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]


Flare ID: 2109230075

Export request pending. [id=JSOC_20230721_360, status=2]
Waiting for 0 seconds...
1 URLs found for download. Full request totalling 8MB


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]


Flare ID: 2110090002

Export request pending. [id=JSOC_20230721_431, status=2]
Waiting for 0 seconds...
1 URLs found for download. Full request totalling 8MB


...

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]


Flare ID: 2204184714

Export request pending. [id=JSOC_20230721_454, status=2]
Waiting for 0 seconds...
1 URLs found for download. Full request totalling 8MB


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]


Flare ID: 2208271589

Export request pending. [id=JSOC_20230721_457, status=2]
Waiting for 0 seconds...
1 URLs found for download. Full request totalling 8MB


Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]